|<h2>Substack post:</h2>|<h1><a href="https://thepalindrome.org/p/the-anatomy-of-the-least-squares-ab5" target="_blank">Least squares part 2: explorations in simulations</a></h1>|
|-|:-:|
|<h2>Teacher:<h2>|<h1>Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h1>|

<br>

<i>Using the code without reading the post may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
### Run this cell only if you're using "dark mode"

# svg plots (higher-res)
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

plt.rcParams.update({
    'figure.facecolor': '#383838',
    'figure.edgecolor': '#383838',
    'axes.facecolor':   '#383838',
    'axes.edgecolor':   '#DDE2F4',
    'axes.labelcolor':  '#DDE2F4',
    'xtick.color':      '#DDE2F4',
    'ytick.color':      '#DDE2F4',
    'text.color':       '#DDE2F4',
    'axes.spines.right': False,
    'axes.spines.top':   False,
    'axes.titleweight': 'bold',
    'axes.labelweight': 'bold',
})

# Simulating regression data

In [ ]:
# 1) coefficients for linking the IV to the DV
B0 = 20  # intercept in happiness units
B1 =  3  # coefficient for change in happiness by punk shows attended

# 2) number of observations
N = 135

# 3) the independent variable
punkshows = np.random.randint(low=0,high=20,size=N)

# 4) and the noise
noise = np.random.normal(0,15,N)

# 5) and now put it together to simulate the data
happiness = B0 + B1*punkshows + noise
happiness[happiness<0] = 0
happiness[happiness>100] = 100


# visualization
plt.figure(figsize=(8,5))

plt.plot(punkshows,happiness,'ko',markerfacecolor=(.9,.9,.9),markersize=10)
plt.gca().set(xlabel='Punk shows attended',ylabel='Life happiness',xticks=range(0,int(punkshows.max())+4,4),
              title='Scatter plot of happiness by punk shows')

plt.tight_layout()

from google.colab import files
plt.savefig('01.svg', format='svg', bbox_inches='tight')

# 3. Trigger the download
files.download('01.svg')
plt.show()

In [ ]:
### run the regression

# 1) organize the IVs into a design matrix
designMatrix = np.vstack((
    np.ones(N,),  # intercept
    punkshows     # IV
    )).T

# 2) list of labels for model output
IVnames = ['Intercept','PunkShows']

# 3) least-squares
betas = np.linalg.lstsq(designMatrix,happiness,rcond=None)[0]

# 4) predicted data and residuals
yHat = designMatrix@betas
resid = happiness - yHat

# and print a summary of the results
print('           | Sim. | beta')
print('-----------+------+------')
print(f' Intercept |  {B0:2}  | {betas[0]:5.2f}' )
print(f' PunkShows |  {B1:2}  | {betas[1]:5.2f}'  )

# Visualizing the regression data

In [ ]:
_,axs = plt.subplots(2,2,figsize=(12,7))

axs[0,0].plot(punkshows,happiness,'wh',markerfacecolor=[.7,.9,.7,.6],markersize=10,label='Observed')
axs[0,0].plot(punkshows,yHat,'m',linewidth=2,label='Predicted')
axs[0,0].set(xlabel='Number of punk shows',ylabel='Happiness',title='1) Observed and predicted data')
axs[0,0].legend()

axs[0,1].plot(happiness,yHat,'wo',markerfacecolor=[.9,.7,.7,.6],markersize=10)
axs[0,1].set(xlabel='Observed happiness',ylabel='Predicted happiness',
             title=f'2) Observed vs. predicted: r = {np.corrcoef(happiness,yHat)[0,1]:.2f}')

axs[1,0].plot(resid,yHat,'ws',markerfacecolor=[.7,.7,.9,.6],markersize=8)
axs[1,0].set(xlabel='Residuals',ylabel='Predicted happiness',
             title=f'3) Resid vs pred: r = {np.corrcoef(resid,yHat)[0,1]:.2f}')

axs[1,1].hist(resid,bins='fd',edgecolor='k',color=[.7,.7,.9])
axs[1,1].set(xlabel='Residuals',ylabel='Counts',title='4) Distribution of residuals')


plt.tight_layout()

plt.savefig('02.svg', format='svg', bbox_inches='tight')

# 3. Trigger the download
files.download('02.svg')
plt.show()

# Adjusted R-squared (R2)

In [ ]:
# "non-adjusted" R2
num = np.sum( (happiness-yHat)**2 )
den = np.sum( (happiness-happiness.mean())**2 )
R2 = 1 - num/den
corrcoef2 = np.corrcoef(happiness,yHat)[0,1]**2

# adjusted R2
num = (1-R2) * (N-1)
den = N - designMatrix.shape[1]
adj_R2 = 1 - num/den

print(f'Regular R2  = {R2:.6f}')
print(f'r(yHat,y)^2 = {corrcoef2:.6f}')
print(f'Adjusted R2 = {adj_R2:.6f}')

# Multivariable regression

In [ ]:
# 1) coefficients for linking the IV to the DV
B0 = 10  # intercept in happiness units
B1 =  3  # coefficient for change in happiness by punk shows attended
B2 = .8  # coefficient for change in happiness by age

# 2) number of observations
N = 88

# 3) the independent variable
punkshows = np.random.randint(low=0,high=15,size=N)
age = np.random.uniform(18,55,N)

# 4) and the noise
noise = np.random.normal(0,15,N)

# 5) and now put it together to simulate the data
happiness = B0 + B1*punkshows + B2*age + noise
happiness[happiness<0] = 0
happiness[happiness>100] = 100

In [ ]:
# visualization
fig,axs = plt.subplots(1,2,figsize=(12,4))

# min-max scaling for marker size
ageScaled = (age-age.min()) / (age.max()-age.min())

# normalization for mapping age to colors
cmap = mpl.cm.plasma
norm = mpl.colors.Normalize(vmin=age.min(),vmax=age.max())

# draw the scatter plot
h = axs[0].scatter(punkshows,happiness,s=100*ageScaled,color=cmap(norm(age)),
                   edgecolor='w',linewidth=.3,marker='o')
axs[0].set(xlabel='Punk shows attended',ylabel='Life happiness',xticks=range(0,int(punkshows.max())+4,4),
              title='Showing age by marker size and color')

# and the color bar
sm = mpl.cm.ScalarMappable(cmap=cmap, norm=norm)
cbar = plt.colorbar(sm,ax=axs[0],pad=.02)
cbar.set_label('Age (years)')


# and draw the data split by median age
age_median = np.median(age)
axs[1].plot(punkshows[age<age_median],happiness[age<age_median],'wo',markerfacecolor=[.9,.3,.3,.7],markersize=10,label='Younger')
axs[1].plot(punkshows[age>=age_median],happiness[age>=age_median],'ws',markerfacecolor=[.3,.3,.9,.7],markersize=10,label='Older')
axs[1].legend()
axs[1].set(xlabel='Punk shows attended',ylabel='Life happiness',xticks=range(0,int(punkshows.max())+4,4),
              title='Median split by age')

plt.tight_layout()

plt.savefig('03.svg', format='svg', bbox_inches='tight')

# 3. Trigger the download
files.download('03.svg')
plt.show()

In [ ]:
age_median

In [ ]:
### run the regression

# organize the IVs into a design matrix
designMatrix = np.vstack((
    np.ones(N,),  # intercept
    punkshows,    # IV
    age
    )).T

# list of labels for model output
IVnames = ['Intercept','PunkShows','Age']

# least-squares
betas = np.linalg.lstsq(designMatrix,happiness,rcond=None)[0]

# predicted data and residuals
yHat = designMatrix@betas
resid = happiness - yHat

# and print a summary of the results
print('           | Sim. | beta')
print('-----------+------+------')
print(f' Intercept |  {B0:2}  | {betas[0]:5.2f}' )
print(f' PunkShows |  {B1:2}  | {betas[1]:5.2f}' )
print(f'       Age | {B2:2}  | {betas[2]:5.2f}' )

# Visualizing the regression data

In [ ]:
_,axs = plt.subplots(2,2,figsize=(12,7))

axs[0,0].plot(punkshows,happiness,'wh',markerfacecolor=[.7,.9,.7,.6],markersize=10,label='Observed')
axs[0,0].plot(punkshows,yHat,'wo',markerfacecolor=[.7,.1,.7,.6],label='Predicted')

for i in range(N):
  axs[0,0].plot([punkshows[i],punkshows[i]],[happiness[i],yHat[i]],'--',color='gray',zorder=-4)


axs[0,0].set(xlabel='Number of punk shows',ylabel='Happiness',title='1) Observed and predicted data')
axs[0,0].legend()

axs[0,1].plot(happiness,yHat,'wo',markerfacecolor=[.9,.7,.7,.6],markersize=10)
axs[0,1].set(xlabel='Observed happiness',ylabel='Predicted happiness',
             title=f'2) Observed vs. predicted: r = {np.corrcoef(happiness,yHat)[0,1]:.2f}')

axs[1,0].plot(resid,yHat,'ws',markerfacecolor=[.7,.7,.9,.6],markersize=8)
axs[1,0].set(xlabel='Residuals',ylabel='Predicted happiness',
             title=f'3) Resid vs pred: r = {np.corrcoef(resid,yHat)[0,1]:.2f}')

axs[1,1].hist(resid,bins='fd',edgecolor='k',color=[.7,.7,.9])
axs[1,1].set(xlabel='Residuals',ylabel='Counts',title='4) Distribution of residuals')


plt.tight_layout()

plt.savefig('04.svg', format='svg', bbox_inches='tight')

# 3. Trigger the download
files.download('04.svg')
plt.show()

In [ ]:
# 1) list of sample sizes (log spaced)
sampleSizes = np.logspace(np.log10(5),np.log10(2000),50).astype(int)

adj_R2 = np.zeros(len(sampleSizes))
R2 = np.zeros(len(sampleSizes))

for idx,N in enumerate(sampleSizes):

  # 2) simulate the data
  punkshows = np.random.randint(low=0,high=15,size=N)
  age = np.random.uniform(18,55,N)
  happiness = B0 + B1*punkshows + B2*age + np.random.normal(0,15,N)
  happiness[happiness<0] = 0
  happiness[happiness>100] = 100

  # 3) build the model
  designMatrix = np.vstack((np.ones(N,),punkshows,age)).T

  # 4) least-squares and get adjR2
  betas = np.linalg.lstsq(designMatrix,happiness,rcond=None)[0]
  yHat = designMatrix@betas
  R2[idx] = np.corrcoef(happiness,yHat)[0,1]**2
  adj_R2[idx] = 1 - (1-R2[idx])*(N-1) / (N-designMatrix.shape[1])


In [ ]:
plt.figure(figsize=(10,4))
plt.plot(sampleSizes,R2,'ks',markersize=12,markerfacecolor=[.9,.7,.7,.7],label='R$^2$')
plt.plot(sampleSizes,adj_R2,'ko',markersize=10,markerfacecolor=[.7,.9,.7,.7],label='$adj.$R$^2$')

plt.legend(fontsize=15)
plt.gca().set(xlabel='Sample size',ylabel='R$^2$',xscale='log')

plt.savefig('05.svg', format='svg', bbox_inches='tight')

# 3. Trigger the download
files.download('05.svg')
plt.show()